<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Test" data-toc-modified-id="Test-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Test</a></span></li><li><span><a href="#--WAITING--" data-toc-modified-id="--WAITING---5"><span class="toc-item-num">5&nbsp;&nbsp;</span>--WAITING--</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Summary</a></span><ul class="toc-item"><li><span><a href="#Load" data-toc-modified-id="Load-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Load</a></span><ul class="toc-item"><li><span><a href="#Train" data-toc-modified-id="Train-6.1.1"><span class="toc-item-num">6.1.1&nbsp;&nbsp;</span>Train</a></span></li><li><span><a href="#Test" data-toc-modified-id="Test-6.1.2"><span class="toc-item-num">6.1.2&nbsp;&nbsp;</span>Test</a></span></li></ul></li><li><span><a href="#Summarize-misassemblies" data-toc-modified-id="Summarize-misassemblies-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Summarize misassemblies</a></span></li></ul></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* Create a test datset with 30 assembly replicates
  * same as for test (n100-r30) in the Version 1 manuscript

# Var

In [1]:
work_dir = '/ebio/abt3_projects/databases_no-backup/DeepMAsED/version2/deepmased-sm/n100-r30/'

threads = 12

# Init

In [2]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(data.table)
library(dtplyr)
library(LeyLabRMisc)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




In [3]:
df.dims()
set.seed(8473)
make_dir(work_dir)

Created directory: /ebio/abt3_projects/databases_no-backup/DeepMAsED/version2/deepmased-sm/n100-r30/ 


# Test

In [5]:
make_dir(file.path(work_dir, 'test'))
config_file = file.path(work_dir, 'test', 'config.yaml')
cat_file(config_file)

Directory already exists: /ebio/abt3_projects/databases_no-backup/DeepMAsED/version2/deepmased-sm/n100-r30//test 
# Input table
genomes_file: /ebio/abt3_projects/databases_no-backup/DeepMAsED/GTDB_ref_genomes/DeepMAsED_GTDB_genome-refs_test_n100.tsv

# Output directory
output_dir: /ebio/abt3_projects/databases_no-backup/DeepMAsED/version2/deepmased-sm/n100-r30/test/

# Temporary output directory (/dev/shm/ for shared memory)
tmp_dir: /ebio/abt3_scratch/

# Email for notifications ("Skip" to skip)
email: nyoungblut@tuebingen.mpg.de

# software parameters
## Use "Skip" to skip  steps. If no params for rule, use ""
## `reps:` is the number of replicate simulations
## If `keep_reads: True`, then simulated reads are kept
params:
  # simulating metagenomes
  reps: 30
  MGSIM:
    genome_download: ""
    community_richness:
      - 1
    reads: --art-paired --art-mflen 250 --art-sdev 10
    read_depth:
      - 1000000
  keep_reads: True #False
  # coverage
  nonpareil: -T kmer
  nonpareil_sum

**DeepMAsED-SM run**

```
(DeepMAsEDv2) @ rick:/ebio/abt3_projects/DeepMAsED/version2/deepmased/DeepMAsED-SM
$ screen -L -S DM-SM-n100-r30-test ./snakemake_sge.sh /ebio/abt3_projects/databases_no-backup/DeepMAsED/version2/deepmased-sm/n100-r30/test/config.yaml 20
```

# --WAITING--

# Summary

## Load

In [ ]:
#' function to read in feature tables
read_feature_table = function(infile){
    assembler = infile %>% dirname %>% basename
    seq_depth = infile %>% dirname %>% dirname %>% basename
    sim_rep = infile %>% dirname %>% dirname %>% dirname %>% basename
    richness = infile %>% dirname %>% dirname %>% dirname %>% dirname %>% basename
    #cmd = glue::glue('gunzip -c {file} | head -n 100000', file=infile)  # <-debug
    cmd = glue::glue('gunzip -c {file}', file=infile)
    data.table::fread(cmd=cmd, sep='\t') %>%
        as_tibble %>%
        mutate(seq_depth = seq_depth,
               sim_rep = sim_rep,
               richness = richness)
}

In [ ]:
feats = list()

### Train

In [ ]:
F = list_files(file.path(work_dir, 'train', 'features'), 'features.tsv.gz')
F %>% length %>% print
F %>% head

In [ ]:
# reading in tables
doParallel::registerDoParallel(threads)
feats[['train']] = F %>%
    #head(n=3) %>%   # <-debug
    as.list %>%
    plyr::llply(read_feature_table, .parallel=TRUE) %>%
    data.table::rbindlist(use.names=TRUE) 

feats[['train']] 

### Test

In [ ]:
F = list_files(file.path(work_dir, 'test', 'features'), 'features.tsv.gz')
F %>% length %>% print
F %>% head

In [ ]:
# reading in tables
doParallel::registerDoParallel(threads)
feats[['test']]  = F %>%
    head(n=3) %>%   # <-debug
    as.list %>%
    plyr::llply(read_feature_table, .parallel=TRUE) %>%
    data.table::rbindlist(use.names=TRUE) 

feats[['test']]

In [ ]:
# combining
feats = data.table::rbindlist(feats, use.names=TRUE, fill=FALSE, idcol='dataset') %>%
    lazy_dt
feats

## Summarize misassemblies

In [ ]:
# summarizing
feats_s = feats %>%
    distinct(dataset, contig, assembler, seq_depth, sim_rep, richness, Extensive_misassembly) %>%
    group_by(dataset, contig, assembler, seq_depth, sim_rep, richness) %>%
    summarize(n_contigs_mis = sum(Extensive_misassembly),
              n_contigs = n()) %>%
    as_tibble 

feats_s

In [ ]:
# number of contigs
p = feats_s %>%
    ggplot(aes(assembler, n_contigs, color=dataset)) +
    geom_boxplot() +
    labs(x='Assembler', y='No. of contigs') +
    theme_bw()

plot(p)

In [ ]:
# number of contigs
p = feats_s %>%
    mutate(perc_misasmbl = n_contigs_mis / n_contigs * 100) %>%
    ggplot(aes(assembler, perc_misasmbl, color=dataset)) +
    geom_boxplot() +
    labs(x='Assembler', y='% misassembled contigs') +
    theme_bw()

plot(p)

In [ ]:
send_email('http://127.0.0.1:9708/notebooks/notebooks/06_version2/01_simulation_datasets/00_n1000/00_n1000_r30_train-test.ipynb')

# sessionInfo

In [ ]:
sessionInfo()